In [13]:
from dotenv import load_dotenv
import os

load_dotenv()


True

In [1]:
import torch
print("GPU available:", torch.cuda.is_available())

GPU available: True


In [ ]:
from transformers import pipeline
import gradio as gr

# pipe = pipeline(model="Siripat/whisper-small-th", device=0)
lang = "th"
task = "transcribe"
device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = pipeline(
    task="automatic-speech-recognition",
    model="nectec/Pathumma-whisper-th-large-v3",
    torch_dtype='auto',
    device=device,
)
pipe.model.config.forced_decoder_ids = pipe.tokenizer.get_decoder_prompt_ids(language=lang, task=task)
text = pipe("audio_path.wav")["text"]
def transcribe(audio):
    text = pipe(audio)["text"]
    return text

iface = gr.Interface(
    fn=transcribe,
    inputs=gr.Audio(type="filepath"),
    outputs="text",
    title="Whisper Small thai",
    description="Realtime demo for thai speech recognition using a fine-tuned Whisper small model.",
)

iface.launch()

Device set to use cuda


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


c:\Users\sirip\Anaconda3\envs\elasticsearch\Lib\site-packages\transformers\models\whisper\generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


In [ ]:
import torch
import librosa
from transformers import AutoModel

device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModel.from_pretrained(
    "nectec/Pathumma-llm-audio-1.0.0",
    torch_dtype='auto',
    lora_infer_mode=True,
    init_from_scratch=True,
    trust_remote_code=True
)
model = model.to(device)

prompt = "ถอดเสียงเป็นข้อความ"
audio_path = "audio_path.wav"
audio, sr = librosa.load(audio_path, sr=16000)

model.eval()
with torch.no_grad():
  response = model.generate(
        raw_wave=audio,
        prompts=prompt,
        device=device,
        max_new_tokens=200,
        repetition_penalty=1.0,
)
print(response[0])
